In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os, sys
import pandas as pd
from dask import bag as db
from dask_k8 import DaskCluster
from dask.distributed import Client

In [4]:
sys.path.append('../')

In [5]:
from sanity_check.contents.s3_data import S3_CANONICAL_DATA_BUCKET
from sanity_check.contents.checks import run_checks_canonical

In [6]:
from sanity_check.contents.s3_data import list_files_rebuilt, fetch_issue_ids_rebuilt
from sanity_check.contents.s3_data import S3_CANONICAL_DATA_BUCKET, fetch_issue_ids
from sanity_check.contents.s3_data import S3_REBUILT_DATA_BUCKET, fetch_issues
# from sanity_check.contents.sync import check_sync_db, configure_db_ingestion
# from sanity_check.contents.sync import check_sync_rebuilt
from sanity_check.contents.checks import run_checks_canonical, find_duplicated_content_item_IDs

In [7]:
from impresso_commons.utils.kube import (make_scheduler_configuration,
                                         make_worker_configuration)

In [8]:
docker_image_uri = "ic-registry.epfl.ch/dhlab/impresso_data-sanity-check:v1"
# docker_image_uri = "ic-registry.epfl.ch/dhlab/impresso_pycommons:v1"

In [9]:
cluster = DaskCluster(
        namespace="dhlab",
        cluster_id="impresso-sanitycheck",
        scheduler_pod_spec=make_scheduler_configuration(),
        worker_pod_spec=make_worker_configuration(
            docker_image=docker_image_uri,
            memory="1G"
        )
    )

In [10]:
cluster.create()
cluster.scale(50, blocking=False)

Scheduler: tcp://10.90.47.29:18197
Dashboard: http://10.90.47.29:31869


In [45]:
cluster.close()

In [11]:
dask_client = cluster.make_dask_client()

In [12]:
# dask_client = Client()

In [16]:
dask_client

Client Scheduler: tcp://10.90.47.29:18197 Dashboard: http://10.90.47.29:8787/status,Cluster Workers: 17 Cores: 17 Memory: 17.00 GB


In [18]:
dask_client.get_versions(check=True)

{'scheduler': {'host': (('python', '3.7.3.final.0'),
   ('python-bits', 64),
   ('OS', 'Linux'),
   ('OS-release', '4.15.0-43-generic'),
   ('machine', 'x86_64'),
   ('processor', ''),
   ('byteorder', 'little'),
   ('LC_ALL', 'C.UTF-8'),
   ('LANG', 'C.UTF-8'),
   ('LOCALE', 'en_US.UTF-8')),
  'packages': {'required': (('dask', '2.3.0'),
    ('distributed', '2.3.2'),
    ('msgpack', '0.6.1'),
    ('cloudpickle', '1.2.1'),
    ('tornado', '6.0.3'),
    ('toolz', '0.10.0')),
   'optional': (('numpy', '1.16.2'),
    ('pandas', '0.24.2'),
    ('bokeh', '1.3.4'),
    ('lz4', None),
    ('dask_ml', None),
    ('blosc', '1.8.1'))}},
 'workers': {'tcp://10.233.108.12:34027': {'host': (('python',
     '3.7.3.final.0'),
    ('python-bits', 64),
    ('OS', 'Linux'),
    ('OS-release', '4.15.0-43-generic'),
    ('machine', 'x86_64'),
    ('processor', ''),
    ('byteorder', 'little'),
    ('LC_ALL', 'C.UTF-8'),
    ('LANG', 'C.UTF-8'),
    ('LOCALE', 'en_US.UTF-8')),
   'packages': {'required': (

In [20]:
!s3cmd ls 

2018-12-14 08:44  s3://TRANSFER
2019-09-24 08:34  s3://ca-canonical
2019-09-24 08:34  s3://ca-clef-canonical
2019-09-24 08:34  s3://ca-clef-rebuilt
2019-09-24 08:34  s3://ca-rebuilt
2018-11-05 08:32  s3://canonical-rebuilt
2019-06-28 09:27  s3://evenized-canonical-rebuilt-new-bnl
2019-06-10 19:34  s3://evenized-canonical-rebuilt-pubrelease
2019-06-11 10:19  s3://evenized-light-canonical-rebuilt-pubrelease
2019-05-01 19:24  s3://impresso-image-labels
2019-03-25 12:19  s3://impresso-images
2019-01-30 14:47  s3://impresso-logs
2018-11-22 10:05  s3://impresso-public
2018-11-20 15:11  s3://nzz-data-test-temporary
2019-01-22 15:15  s3://original-canonical-compressed
2018-06-06 08:17  s3://original-canonical-data
2019-06-07 08:33  s3://original-canonical-data-publrelease
2019-06-16 09:39  s3://original-canonical-fixed
2019-09-12 07:59  s3://original-canonical-staging
2019-03-28 12:38  s3://passim-rebuilt
2018-06-06 08:17  s3://processed-canonical-data
2019-05-10 07:49  s3://snapshots


## Check for long articles

In [16]:
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT, fixed_s3fs_glob
from dask import bag as db
import json

In [163]:
rebuilt_bucket = "s3://canonical-rebuilt-release"
newspaper = "indeplux"

In [164]:
rebuilt_files = fixed_s3fs_glob(f'{rebuilt_bucket}/{newspaper}/*.bz2')

In [165]:
len(rebuilt_files)

61

In [166]:
from collections import Counter

def find_ngrams(text, ngram_size, threshold):
    
    # create ngrams of a given size
    sentence = text.split()
    N = ngram_size
    grams = [
        "/".join(sentence[i:i+N])
        for i in range(len(sentence)-N+1)
    ]
    
    # check for long repeated chunks of text
    filtered_ngrams = []
    for gram, count in Counter(grams).items():
        if count >= threshold:
            filtered_ngrams.append((gram, count))
    
    return filtered_ngrams

In [167]:
def check_repeated_text(content_item):

    if "ft" not in content_item:
        return None

    text = content_item['ft']
    repeated_long_grams = find_ngrams(text, 100, 2)

    if repeated_long_grams:
        return {
            "ci_id": content_item['id'],
            "n_repeated_grams": len(repeated_long_grams)
        }
    else:
        return None

In [ ]:
from dask import bag as db
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT

db.read_text(
    "s3://canonical-rebuilt-release/JDG/JDG-1998.jsonl.bz2",
    storage_options=IMPRESSO_STORAGEOPT
).map(
    json.loads
).take(1)

In [168]:
long_content_items = db.read_text(
    rebuilt_files,
    storage_options=IMPRESSO_STORAGEOPT
).map(
    json.loads
).filter(
    lambda i: i is not None and "ft" in i
).map(
    lambda i: {"id":i["id"], "ft":i["ft"]}
).map(
    check_repeated_text
).filter(
    lambda x: x is not None
).compute()

In [31]:
dask_client.cancel(content_items)

In [28]:
dask_client.get_futures_error(content_items)

(<function distributed.worker.execute_task(task)>,
 ((<function dask.bag.core.reify(seq)>,
   (<function dask.bag.core.map_chunk(f, iters, iter_kwarg_keys=None, kwargs=None)>,
    <function __main__.<lambda>(i)>,
    [(filter,
      <function __main__.<lambda>(i)>,
      (<function dask.bag.core.map_chunk(f, iters, iter_kwarg_keys=None, kwargs=None)>,
       <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>,
       [(<function _operator.getitem>,
         "('split-3e91f7f76ee7a3e6d778ccce528ada99', 14)",
         0)],
       None,
       {}))],
    None,
    {})),),
 {},
 ("('split-3e91f7f76ee7a3e6d778ccce528ada99', 14)",))

In [169]:
long_content_items = content_items.map(
    check_repeated_text
).filter(lambda x: x is not None).compute()

In [ ]:
cluster.close()

In [170]:
len(long_content_items)

29

In [171]:
long_content_items

[{'ci_id': 'indeplux-1872-04-06-a-i0005', 'n_repeated_grams': 95},
 {'ci_id': 'indeplux-1874-01-24-a-i0001', 'n_repeated_grams': 341},
 {'ci_id': 'indeplux-1876-10-28-a-i0001', 'n_repeated_grams': 28},
 {'ci_id': 'indeplux-1876-10-28-a-i0011', 'n_repeated_grams': 172},
 {'ci_id': 'indeplux-1876-10-29-a-i0001', 'n_repeated_grams': 102},
 {'ci_id': 'indeplux-1879-08-20-a-i0007', 'n_repeated_grams': 193},
 {'ci_id': 'indeplux-1879-11-07-a-i0008', 'n_repeated_grams': 128},
 {'ci_id': 'indeplux-1879-11-12-a-i0002', 'n_repeated_grams': 259},
 {'ci_id': 'indeplux-1880-06-06-a-i0001', 'n_repeated_grams': 178},
 {'ci_id': 'indeplux-1881-09-24-a-i0014', 'n_repeated_grams': 502},
 {'ci_id': 'indeplux-1882-05-12-a-i0008', 'n_repeated_grams': 39},
 {'ci_id': 'indeplux-1882-05-23-a-i0009', 'n_repeated_grams': 24},
 {'ci_id': 'indeplux-1882-07-23-a-i0008', 'n_repeated_grams': 214},
 {'ci_id': 'indeplux-1882-07-21-a-i0015', 'n_repeated_grams': 319},
 {'ci_id': 'indeplux-1884-04-11-a-i0005', 'n_repeate

In [74]:
ci_id = long_content_items[1]['ci_id']

In [172]:
ci_id = "indeplux-1890-05-10-a-i0015"

In [151]:
from impresso_commons.utils.s3 import alternative_read_text

In [152]:
def get_issue(issue_id, bucket_name):
    newspaper, year = issue_id.split('-')[:2]
    file = f'{bucket_name}/{newspaper}/issues/{newspaper}-{year}-issues.jsonl.bz2'
    print(file)
    return db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == issue_id
    ).compute()

In [153]:
def get_page(page_id, bucket_name):
    newspaper, year = page_id.split('-')[:2]
    issue = "-".join(page_id.split('-')[:-1])
    file = f'{bucket_name}/{newspaper}/pages/{newspaper}-{year}/{issue}-pages.jsonl.bz2'
    print(file)
    data = db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == page_id
    ).compute()
    
    if len(data) > 0:
        return data[0]
    else:
        return None

In [154]:
def get_content_item(ci_id, bucket_name):
    newspaper, year = ci_id.split('-')[:2]
    file = f'{bucket_name}/{newspaper}/{newspaper}-{year}.jsonl.bz2'
    
    data = db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == ci_id
    ).compute()
    
    if len(data) > 0:
        return data[0]
    else:
        return None

In [173]:
tmp = get_content_item(ci_id, "s3://canonical-rebuilt-release")

reading s3://canonical-rebuilt-release/indeplux/indeplux-1890.jsonl.bz2


In [186]:
tmp['id']

'indeplux-1890-05-10-a-i0015'

In [187]:
tmp['pp']

[3]

In [182]:
page = get_page("indeplux-1890-05-10-a-p0003", "s3://original-canonical-staging")

s3://original-canonical-staging/indeplux/pages/indeplux-1890/indeplux-1890-05-10-a-pages.jsonl.bz2
reading s3://original-canonical-staging/indeplux/pages/indeplux-1890/indeplux-1890-05-10-a-pages.jsonl.bz2


In [183]:
page

{'id': 'indeplux-1890-05-10-a-p0003',
 'cdt': '2019-10-16 11:31:46',
 'r': [{'c': [192, 266, 755, 174],
   'p': [{'c': [192, 266, 755, 174],
     'l': [{'c': [198, 270, 745, 43],
       't': [{'c': [198, 277, 60, 31], 'tx': 'que'},
        {'c': [283, 270, 43, 31], 'tx': 'les'},
        {'c': [351, 271, 140, 34], 'tx': 'ouvriers'},
        {'c': [517, 275, 55, 29], 'tx': 'ont'},
        {'c': [596, 270, 186, 43], 'tx': 'consignées'},
        {'c': [805, 270, 77, 31], 'tx': 'dans'},
        {'c': [902, 281, 42, 22], 'tx': 'un'}]},
      {'c': [199, 314, 745, 43],
       't': [{'c': [199, 314, 244, 37], 'tx': 'procès-verbal.'},
        {'c': [493, 317, 49, 31], 'tx': 'On'},
        {'c': [579, 316, 113, 42], 'tx': 'espère'},
        {'c': [728, 327, 61, 28], 'tx': 'que'},
        {'c': [826, 315, 42, 34], 'tx': 'M.'},
        {'c': [904, 315, 40, 31], 'tx': 'de'}]},
      {'c': [198, 356, 748, 47],
       't': [{'c': [198, 356, 137, 35], 'tx': 'Wendel'},
        {'c': [360, 362, 188, 41]

In [184]:
issue = get_issue("indeplux-1890-05-10-a", "s3://original-canonical-staging")

s3://original-canonical-staging/indeplux/issues/indeplux-1890-issues.jsonl.bz2
reading s3://original-canonical-staging/indeplux/issues/indeplux-1890-issues.jsonl.bz2


In [185]:
issue

[{'cdt': '2019-10-16 11:31:46',
  'i': [{'m': {'id': 'indeplux-1890-05-10-a-i0001',
     'l': 'fr',
     'tp': 'article',
     'pp': [1, 3],
     't': 'Dépêches et Dernières Nouvelles. Agence libre.'},
    'l': {'id': 'MODSMD_ARTICLE1',
     'parts': [{'comp_role': 'heading',
       'comp_id': 'ART2-1',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'heading',
       'comp_id': 'ART2-2',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-3',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-4',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-5',
       'comp_fileid': 'ALTO00003',
       'comp_page_no': 3},
      {'comp_role': 'body',
       'comp_id': 'ART2-6',
       'comp_fileid': 'ALTO00003',
       'comp_page_no': 3},
      {'comp_role': 'body',
       'comp

In [177]:
ngrams = find_ngrams(tmp['ft'], 100, 2)

In [188]:
ngrams[:10]

[("un/homme/d’une/habileté/supérieure/et/d’une/audace/extraordinaire,/mais/qui/tenait/un/compte/insuffisant/des/droits/du/prochain;/la/manière/même/dont/s'était/contracté/son/mariage/est/caractéristique/de/ses/procédés/de/gouvernement.»/Au/cours/d’un/voyage/électoral,/alors/qu’il/était/candidat/à/la/présidence,/il/avait/eu/l’occasion/de/voir/dans/un/couvent/une/jeune/fille/parfaitement/belle./Il/s’enquit/de/son/nom,/de/sa/famille/et/écrivit/au/père/que/l’enfant/lui/plaisait/beaucoup/et/qu’il/désirait/l’épouser/quand/son/éducation/sera/terminée./«La/famille/avait/des/prétentions/aristocratiques/;/elle/dédaigna/les/ouvertures/de/Barrios/et",
  2),
 ("homme/d’une/habileté/supérieure/et/d’une/audace/extraordinaire,/mais/qui/tenait/un/compte/insuffisant/des/droits/du/prochain;/la/manière/même/dont/s'était/contracté/son/mariage/est/caractéristique/de/ses/procédés/de/gouvernement.»/Au/cours/d’un/voyage/électoral,/alors/qu’il/était/candidat/à/la/présidence,/il/avait/eu/l’occasion/de/voir/dans/

In [178]:
len(ngrams)

300

## Check canonical (content item IDs)

In [41]:
from sanity_check.contents.checks import check_duplicated_content_item_IDs

In [50]:
from dask import bag
import pandas as pd
from collections import Counter

def l_find_duplicated_content_item_IDs(issue_json: dict) -> list:
    ci_ids = [ci['m']['id'] for ci in issue_json['i']]
    duplicates = []
    for ci_id, count in Counter(ci_ids).items():
        if count > 1:
            duplicates.append(ci_id)
    return duplicates


def l_check_duplicated_content_item_IDs(issue_bag: bag) -> pd.DataFrame:
    duplicates_bag = issue_bag.map(
        l_find_duplicated_content_item_IDs
    ).flatten()

    duplicates = duplicates_bag.map(
        lambda i: {
            "id": i,
            "issue_id": "-".join(i.split('-')[:-1]),
            "newspaper_id": i.split('-')[0],
            "year": int(i.split('-')[1]),
        }
    ).compute()

    if duplicates:
        duplicates_df = pd.DataFrame(duplicates).set_index('id')
    else:
        # there are no duplicates
        duplicates_df = pd.DataFrame(
            columns=['id', 'issue_id', 'newspaper_id', 'year']
        )

    print((
        f'Found {duplicates_df.shape[0]} duplicated '
        'content item IDs, belonging to '
        f'{duplicates_df.newspaper_id.unique().size} journals.'
    ))
    return duplicates_df

In [51]:
canonical_issues_bag = fetch_issues(
    "s3://original-canonical-staging",
    compute=False
).filter(lambda i: len(i) > 0)

# 1) verify that there are not duplicated content item IDs
duplicates_df = l_check_duplicated_content_item_IDs(canonical_issues_bag)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)
Found 0 duplicated content item IDs, belonging to 0 journals.


In [52]:
duplicates_df.head()

,id,issue_id,newspaper_id,year


In [27]:
pd.DataFrame(columns=['id', 'issue_id', 'newspaper_id', 'year'])

,id,issue_id,newspaper_id,year


## Check canonical (page IDs)

In [60]:
from sanity_check.contents.checks import check_inconsistent_page_ids

In [61]:
check_inconsistent_page_ids?

In [62]:
inconsistencies_df = check_inconsistent_page_ids("s3://original-canonical-staging")

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)
Fetching page IDs from issues
Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)


TypeError: 'NoneType' object is not iterable

### Pages from issue JSON

In [57]:
canonical_bucket = "s3://original-canonical-staging"

In [64]:
from collections import Counter

In [65]:
issue_bag = fetch_issues(
    canonical_bucket,
    compute=False
)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)


In [66]:
page_ids_from_issues = issue_bag.map(lambda i: i['pp']).flatten().compute()

In [67]:
len(page_ids_from_issues)

5526792

In [68]:
len(set(page_ids_from_issues))

5510159

In [69]:
duplicates = []
for page_id, count in Counter(page_ids_from_issues).items():
    if count > 1:
        duplicates.append(page_id)

In [70]:
len(duplicates)

16633

In [71]:
duplicates

['JDG-1973-07-07-a-p0001',
 'JDG-1973-07-07-a-p0002',
 'JDG-1973-07-07-a-p0003',
 'JDG-1973-07-07-a-p0004',
 'JDG-1973-07-07-a-p0005',
 'JDG-1973-07-07-a-p0006',
 'JDG-1973-07-07-a-p0007',
 'JDG-1973-07-07-a-p0008',
 'JDG-1973-07-07-a-p0009',
 'JDG-1973-07-07-a-p0010',
 'JDG-1973-07-07-a-p0011',
 'JDG-1973-07-07-a-p0012',
 'JDG-1973-07-07-a-p0013',
 'JDG-1973-07-07-a-p0014',
 'JDG-1973-07-07-a-p0015',
 'JDG-1973-07-07-a-p0016',
 'JDG-1973-07-07-a-p0017',
 'JDG-1973-07-07-a-p0018',
 'JDG-1973-07-07-a-p0019',
 'JDG-1973-07-07-a-p0020',
 'JDG-1973-07-07-a-p0021',
 'JDG-1973-07-07-a-p0022',
 'JDG-1973-07-07-a-p0023',
 'JDG-1973-07-07-a-p0024',
 'JDG-1973-07-20-a-p0001',
 'JDG-1973-07-20-a-p0002',
 'JDG-1973-07-20-a-p0003',
 'JDG-1973-07-20-a-p0004',
 'JDG-1973-07-20-a-p0005',
 'JDG-1973-07-20-a-p0006',
 'JDG-1973-07-20-a-p0007',
 'JDG-1973-07-20-a-p0008',
 'JDG-1973-07-20-a-p0009',
 'JDG-1973-07-20-a-p0010',
 'JDG-1973-07-20-a-p0011',
 'JDG-1973-07-20-a-p0012',
 'JDG-1973-07-20-a-p0013',
 

### Pages from pages JSON

In [14]:
dask_client

Client Scheduler: tcp://10.90.47.34:23227 Dashboard: http://10.90.47.34:8787/status,Cluster Workers: 163 Cores: 163 Memory: 815.00 GB


In [28]:
import json
from sanity_check.contents.s3_data import list_pages, list_newspapers
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT, fixed_s3fs_glob, alternative_read_text
from impresso_commons.utils import chunk

In [29]:
page_files = list_pages('s3://original-canonical-staging/')

Fetching list of newspapers from s3://original-canonical-staging/
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging/ contains 604206 .bz2 files


In [30]:
page_ids = db.from_sequence(page_files, partition_size=200).map(
    alternative_read_text, IMPRESSO_STORAGEOPT
).flatten().map(json.loads).filter(
    lambda i: len(i) > 0
).pluck('id')

In [31]:
page_df = page_ids.map(
        lambda id: {"id": id, "from_pages": True}
    ).to_dataframe().set_index('id').persist()

In [35]:
page_df.shape[0].compute()

6073551

In [73]:
def list_pages(bucket_name=S3_CANONICAL_DATA_BUCKET):
    if bucket_name:
        newspapers = list_newspapers(bucket_name)
    else:
        newspapers = list_newspapers()

    page_files = db.from_sequence(newspapers).map(
        lambda np: fixed_s3fs_glob(
            f"{os.path.join(bucket_name, f'{np}/pages/*')}"
        )
    ).persist()
    print(f'{bucket_name} contains {page_files.count().compute()} .bz2 files')
    return page_files

In [88]:
page_files_bag = list_pages(canonical_bucket)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 78 .bz2 files


In [108]:
page_ids = db.from_sequence(page_files_bag).repartition(6000).map(
    alternative_read_text, IMPRESSO_STORAGEOPT
).flatten().map(json.loads).filter(
    lambda i: len(i) > 0
).pluck('id').compute()

KilledWorker: ("('pluck-cacaa9a0255dfb1635dc70bd7c3ac85b', 2397)", <Worker 'tcp://10.233.64.40:35881', memory: 0, processing: 11>)

In [ ]:
page_ids = db.from_sequence(page_files_bag).map(
    alternative_read_text, IMPRESSO_STORAGEOPT
)

In [96]:
def read_pages(page_files):
    # the read text could be wrapped in a try/except clause
    bag_ids = db.from_sequence(
        alternative_read_text(
            page_files,
            IMPRESSO_STORAGEOPT
        )
    ).map(
        json.loads
    ).filter(lambda i: len(i) > 0).pluck('id')
    return bag_ids.compute()

In [83]:
from random import shuffle

In [89]:
page_files_bag = page_files_bag.flatten().compute()

In [90]:
shuffle(page_files_bag)

In [91]:
page_files_bag[:10]

['s3://original-canonical-staging/FZG/pages/FZG-1966/FZG-1966-03-15-a-pages.jsonl.bz2',
 's3://original-canonical-staging/GAV/pages/GAV-1894/GAV-1894-06-23-a-pages.jsonl.bz2',
 's3://original-canonical-staging/IMP/pages/IMP-1895/IMP-1895-12-13-a-pages.jsonl.bz2',
 's3://original-canonical-staging/EXP/pages/EXP-1963/EXP-1963-05-27-a-pages.jsonl.bz2',
 's3://original-canonical-staging/luxwort/pages/luxwort-1935/luxwort-1935-07-06-a-pages.jsonl.bz2',
 's3://original-canonical-staging/luxwort/pages/luxwort-1949/luxwort-1949-01-14-a-pages.jsonl.bz2',
 's3://original-canonical-staging/NZZ/pages/NZZ-1905/NZZ-1905-08-28-c-pages.jsonl.bz2',
 's3://original-canonical-staging/LCE/pages/LCE-1871/LCE-1871-10-08-a-pages.jsonl.bz2',
 's3://original-canonical-staging/onsjongen/pages/onsjongen-1948/onsjongen-1948-07-16-a-pages.jsonl.bz2',
 's3://original-canonical-staging/NZG/pages/NZG-1863/NZG-1863-03-28-a-pages.jsonl.bz2']

In [92]:
len(page_files_bag)

604206

In [42]:
filtered_pages = [
    page
    for page in page_files_bag
    if "NZZ-" not in page
]

In [43]:
len(filtered_pages)

343208

In [101]:
len(page_files_bag)

604206

In [102]:
%%time
page_ids = []
batch_counter = 0

# divide the list of bz2 files (one per newspaper issue)
# into even chunks of size n. Otherwise, if we keep the
# partitions per newspaper everything blows up for newspapers
# that have hundreds of thousands of issues
for batch in chunk(page_files_bag, 2500):
    batch_counter += 1
    print(f'Processing batch {batch_counter} of size {len(batch)}')
    page_ids += read_pages(batch)
    print(f'{len(page_ids)} page ids collected')
    

Processing batch 1 of size 2500
reading ['s3://original-canonical-staging/FZG/pages/FZG-1966/FZG-1966-03-15-a-pages.jsonl.bz2', 's3://original-canonical-staging/GAV/pages/GAV-1894/GAV-1894-06-23-a-pages.jsonl.bz2', 's3://original-canonical-staging/IMP/pages/IMP-1895/IMP-1895-12-13-a-pages.jsonl.bz2', 's3://original-canonical-staging/EXP/pages/EXP-1963/EXP-1963-05-27-a-pages.jsonl.bz2', 's3://original-canonical-staging/luxwort/pages/luxwort-1935/luxwort-1935-07-06-a-pages.jsonl.bz2', 's3://original-canonical-staging/luxwort/pages/luxwort-1949/luxwort-1949-01-14-a-pages.jsonl.bz2', 's3://original-canonical-staging/NZZ/pages/NZZ-1905/NZZ-1905-08-28-c-pages.jsonl.bz2', 's3://original-canonical-staging/LCE/pages/LCE-1871/LCE-1871-10-08-a-pages.jsonl.bz2', 's3://original-canonical-staging/onsjongen/pages/onsjongen-1948/onsjongen-1948-07-16-a-pages.jsonl.bz2', 's3://original-canonical-staging/NZG/pages/NZG-1863/NZG-1863-03-28-a-pages.jsonl.bz2', 's3://original-canonical-staging/waechtersauer/

TypeError: don't know how to handle uri ['s3://original-canonical-staging/FZG/pages/FZG-1966/FZG-1966-03-15-a-pages.jsonl.bz2', 's3://original-canonical-staging/GAV/pages/GAV-1894/GAV-1894-06-23-a-pages.jsonl.bz2', 's3://original-canonical-staging/IMP/pages/IMP-1895/IMP-1895-12-13-a-pages.jsonl.bz2', 's3://original-canonical-staging/EXP/pages/EXP-1963/EXP-1963-05-27-a-pages.jsonl.bz2', 's3://original-canonical-staging/luxwort/pages/luxwort-1935/luxwort-1935-07-06-a-pages.jsonl.bz2', 's3://original-canonical-staging/luxwort/pages/luxwort-1949/luxwort-1949-01-14-a-pages.jsonl.bz2', 's3://original-canonical-staging/NZZ/pages/NZZ-1905/NZZ-1905-08-28-c-pages.jsonl.bz2', 's3://original-canonical-staging/LCE/pages/LCE-1871/LCE-1871-10-08-a-pages.jsonl.bz2', 's3://original-canonical-staging/onsjongen/pages/onsjongen-1948/onsjongen-1948-07-16-a-pages.jsonl.bz2', 's3://original-canonical-staging/NZG/pages/NZG-1863/NZG-1863-03-28-a-pages.jsonl.bz2', 's3://original-canonical-staging/waechtersauer/

In [98]:
page_ids

[]

In [51]:
assert len(page_ids) == len(set(page_ids))

In [52]:
import pickle

In [54]:
page_pickle_fname = os.path.join(
    "../reports/contents/",
    "pages_ids_from_pages.pkl"
)

with open(page_pickle_fname, "wb") as outfile:
    pickle.dump(page_ids, outfile)

In [72]:
df_page_ids_from_pages = db.from_sequence(page_ids).map(
    lambda id: {"id": id, "from_pages": True}
).to_dataframe().set_index('id').persist()

In [86]:
df_page_ids_from_issues = db.from_sequence(set(page_ids_from_issues)).map(
    lambda id: {"id": id, "from_issues": True}
).to_dataframe().set_index('id').persist()

In [87]:
df_page_ids_from_pages.head()

,from_pages
id,
BDC-1839-01-20-a-p0001,True
BDC-1839-01-20-a-p0002,True
BDC-1839-01-20-a-p0003,True
BDC-1839-01-20-a-p0004,True
BDC-1839-01-20-a-p0005,True


In [88]:
df_page_ids_from_issues.head()

,from_issues
id,
BDC-1839-01-20-a-p0001,True
BDC-1839-01-20-a-p0002,True
BDC-1839-01-20-a-p0003,True
BDC-1839-01-20-a-p0004,True
BDC-1839-01-20-a-p0005,True


In [89]:
df_pages = df_page_ids_from_pages.join(df_page_ids_from_issues, how='outer').compute()

In [96]:
df_pages['newspaper_id'] = df_pages.index.map(lambda z: z.split('-')[0])

In [97]:
df_pages.head()

,from_pages,from_issues,newspaper_id
id,,,
BDC-1839-01-20-a-p0001,True,True,BDC
BDC-1839-01-20-a-p0002,True,True,BDC
BDC-1839-01-20-a-p0003,True,True,BDC
BDC-1839-01-20-a-p0004,True,True,BDC
BDC-1839-01-20-a-p0005,True,True,BDC


In [151]:
df_pages[~(df_pages.from_pages == df_pages.from_issues)].shape

(562294, 3)

In [109]:
df_pages[~(df_pages.from_pages == df_pages.from_issues) & ~(df_pages.newspaper_id=='NZZ')].newspaper_id.unique()

array(['indeplux', 'luxembourg1935', 'luxwort', 'tageblatt'], dtype=object)

In [120]:
df_pages[~(df_pages.from_pages == df_pages.from_issues) & (df_pages.newspaper_id=='tageblatt')]

                             from_pages  from_issues newspaper_id
id                                                               
tageblatt-1913-10-11-a-p0007        NaN         True    tageblatt
tageblatt-1913-10-11-a-p0008        NaN         True    tageblatt
tageblatt-1913-10-11-a-p0009        NaN         True    tageblatt
tageblatt-1913-10-11-a-p0010        NaN         True    tageblatt
tageblatt-1920-05-05-a-p0003        NaN         True    tageblatt
...                                 ...          ...          ...
tageblatt-1947-10-06-a-p0005        NaN         True    tageblatt
tageblatt-1949-05-21-a-p0008        NaN         True    tageblatt
tageblatt-1949-05-21-a-p0009        NaN         True    tageblatt
tageblatt-1949-05-21-a-p0010        NaN         True    tageblatt
tageblatt-1950-10-16-a-p0007        NaN         True    tageblatt

[808 rows x 3 columns]


In [107]:
df_pages[~(df_pages.from_pages == df_pages.from_issues) & ~(df_pages.newspaper_id=='NZZ')].head(20)

,from_pages,from_issues,newspaper_id
id,,,
indeplux-1873-01-10-a-p0006,NaN,True,indeplux
indeplux-1876-02-16-a-p0008,NaN,True,indeplux
indeplux-1878-11-15-a-p0008,NaN,True,indeplux
indeplux-1881-05-05-a-p0006,NaN,True,indeplux
indeplux-1886-04-15-a-p0001,NaN,True,indeplux
indeplux-1886-04-15-a-p0002,NaN,True,indeplux
indeplux-1886-04-16-a-p0001,NaN,True,indeplux
indeplux-1886-04-16-a-p0002,NaN,True,indeplux
indeplux-1886-04-17-a-p0001,NaN,True,indeplux


In [177]:
df_pages.shape

(3812590, 3)

In [123]:
df_pages_no_nzz = df_pages[
    ~(df_pages.from_pages == df_pages.from_issues) & 
    ~(df_pages.newspaper_id=='NZZ')
]

In [181]:
df_pages_no_nzz.groupby(by="newspaper_id").agg({"newspaper_id": len})

,newspaper_id
newspaper_id,
indeplux,22
luxembourg1935,2
luxwort,3
tageblatt,808


In [180]:
", ".join(list(df_pages_no_nzz.newspaper_id.unique()))

'indeplux, luxembourg1935, luxwort, tageblatt'

In [125]:
df_pages_no_nzz.to_csv("../reports/contents/inconsistent_page_ids.csv")

In [104]:
df_pages.to_pickle("../reports/contents/pages_ids_df.pkl")

In [208]:
cluster.close()

In [214]:
dask_client

Client Scheduler: tcp://10.90.47.6:24439 Dashboard: http://10.90.47.6:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client -

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), 

In [41]:
# this remains a mystery...
db.read_text(
    "s3://original-canonical-fixed/NZZ/pages/NZZ-1926/NZZ-1926-09-08-e-pages.jsonl.bz2",
    storage_options=IMPRESSO_STORAGEOPT
).map(json.loads).pluck('id').compute()

FileNotFoundError: original-canonical-fixed/NZZ/pages/NZZ-1926/NZZ-1926-09-08-e-pages.jsonl.bz2

In [34]:
cluster.close()

## Check canonical (duplicated issue IDs)

In [128]:
from dask.distributed import Client

In [129]:
dask_client = Client()

In [24]:
dask_client.get_versions(check=True)

ValueError: Mismatched versions found

blosc
+----------------------------+---------+
|                            | version |
+----------------------------+---------+
| client                     | None    |
| scheduler                  | 1.8.1   |
| tcp://10.233.107.101:36389 | 1.8.1   |
| tcp://10.233.107.14:34051  | 1.8.1   |
| tcp://10.233.107.30:39961  | 1.8.1   |
| tcp://10.233.107.59:45321  | 1.8.1   |
| tcp://10.233.107.5:43523   | 1.8.1   |
| tcp://10.233.107.80:35235  | 1.8.1   |
| tcp://10.233.107.90:35169  | 1.8.1   |
| tcp://10.233.112.150:37595 | 1.8.1   |
| tcp://10.233.112.178:44157 | 1.8.1   |
| tcp://10.233.112.190:43291 | 1.8.1   |
| tcp://10.233.112.209:38223 | 1.8.1   |
| tcp://10.233.112.223:37999 | 1.8.1   |
| tcp://10.233.112.250:34299 | 1.8.1   |
| tcp://10.233.112.255:37955 | 1.8.1   |
| tcp://10.233.116.195:42703 | 1.8.1   |
| tcp://10.233.116.200:43395 | 1.8.1   |
| tcp://10.233.116.245:39281 | 1.8.1   |
| tcp://10.233.116.247:40215 | 1.8.1   |
| tcp://10.2

In [31]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.client - ERROR - Failed t

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent ca

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268,

In [19]:
dask_client

Client Scheduler: tcp://10.90.47.23:31733 Dashboard: http://10.90.47.23:8787/status,Cluster Workers: 100 Cores: 100 Memory: 100.00 GB


In [20]:
from sanity_check.contents.s3_data import fetch_issue_ids, fetch_issue_ids_rebuilt
from sanity_check.contents.checks import check_duplicated_issues_IDs

In [27]:
issue_bag = fetch_issues(
    's3://original-canonical-testing/',
    compute=False
)

Fetching list of newspapers from s3://original-canonical-testing/
original-canonical-testing contains 4 newspapers
s3://original-canonical-testing/ contains 214 .bz2 files
Fetching issue ids from 214 .bz2 files (compute=False)


In [28]:
duplicate_issue_ids = check_duplicated_issues_IDs(issue_bag)

using input issue bag dask.bag<loads-0..., npartitions=214>
12409 issue IDs were fetched
905 duplicated IDs were found


In [30]:
len(duplicate_issue_ids)

905

## Sync rebuilt/canonical

In [191]:
s3_canonical_issues = fetch_issue_ids(
    's3://original-canonical-staging',
    compute=False
)
s3_rebuilt_issues = fetch_issue_ids_rebuilt(
    's3://canonical-rebuilt-release',
    compute=False
)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)
Fetching list of newspapers from s3://canonical-rebuilt-release
canonical-rebuilt-release contains 76 newspapers
s3://canonical-rebuilt-release contains 3039 .bz2 files


In [193]:
s3_rebuilt_data = s3_rebuilt_issues.map(
    lambda i: {'id': i, "in_rebuilt": True}
).to_dataframe().set_index('id').compute()

In [194]:
s3_canonical_data = s3_canonical_issues.map(
    lambda i: {'id': i, "in_canonical": True}
).to_dataframe().set_index('id').compute()

In [195]:
print('Joining the two dataframes')
issue_data = s3_rebuilt_data.join(
    s3_canonical_data,
    how='outer'
)
print('Joining the two dataframes... done')

issue_data['newspaper_id'] = issue_data.index.map(
    lambda i: i.split('-')[0]
)
issue_data['year'] = issue_data.index.map(
    lambda i: int(i.split('-')[1])
)

issues_to_rebuild = issue_data[
    (issue_data.in_canonical.notnull()) &
    (issue_data.in_rebuilt.isnull())
]
issues_to_ingest = issue_data[
    (issue_data.in_canonical.isnull()) &
    (issue_data.in_rebuilt.notnull())
]

Joining the two dataframes
Joining the two dataframes... done


In [205]:
issues_to_rebuild[(issues_to_rebuild.newspaper_id=='luxwort') & (issues_to_rebuild.year == 1852)]

,in_rebuilt,in_canonical,newspaper_id,year
id,,,,
luxwort-1852-07-02-a,NaN,True,luxwort,1852


In [221]:
issue_data.shape

(604769, 4)

other statistics to add (per newspaper):
- number of pages
- number of content items
- number of images
- number of tokens

In [233]:
issue_data.groupby(
    by='newspaper_id'
).agg(
    {'newspaper_id': len}
).rename(
    columns={'newspaper_id': 'n_issues'}
).to_csv(
    '../../impresso-processing/data/corpus-public-release-issues.csv',
    encoding='utf-8'
)

In [234]:
!head ../../impresso-processing/data/corpus-public-release-issues.csv

newspaper_id,n_issues
BDC,23
BLB,110
BNN,1906
CDV,1200
CON,101
DFS,1825
DLE,402
DTT,12970
DVF,876


In [231]:
issues_to_rebuild[
    ~issues_to_rebuild.newspaper_id.isin(['arbeitgeber', 'handelsztg'])
].groupby(by='newspaper_id').agg({'newspaper_id': len}).rename(columns={'newspaper_id': 'n_issues'})

,n_issues
newspaper_id,
DTT,1
EXP,1
luxwort,145


In [196]:
issues_to_rebuild.newspaper_id.unique()

array(['DTT', 'EXP', 'arbeitgeber', 'handelsztg', 'luxwort'], dtype=object)

In [18]:
issues_to_ingest, issues_to_rebuild = check_sync_rebuilt(
    's3://original-canonical-staging',
    's3://canonical-rebuilt-release'
)

Fetching list of newspapers from s3://original-canonical-fixed


KeyboardInterrupt: 

In [227]:
issues_to_ingest.to_pickle('../reports/contents/issues_to_ingest.pkl')

In [228]:
issues_to_rebuild.to_pickle('../reports/contents/issues_to_rebuild.pkl')

In [229]:
issues_to_ingest.head()

,in_rebuilt,in_canonical,newspaper_id,year
id,,,,


In [230]:
issues_to_rebuild.newspaper_id.unique()

array(['DTT', 'EXP', 'arbeitgeber', 'handelsztg', 'luxwort'], dtype=object)

## Configure rebuild

In [35]:
dask_client

Client Scheduler: tcp://10.90.47.29:28764 Dashboard: http://10.90.47.29:8787/status,Cluster Workers: 100 Cores: 100 Memory: 2.00 TB


In [55]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client -

concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/pycommons/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell

In [32]:
from sanity_check.contents.sync import sync_rebuilt
from sanity_check.contents.sync import configure_db_ingestion, configure_ingestion, configure_rebuild

In [37]:
s3_canonical = "s3://original-canonical-staging"
s3_rebuilt = "s3://passim-rebuilt"

In [39]:
issues_to_ingest, issues_to_rebuild = sync_rebuilt(
    s3_canonical,
    s3_rebuilt
)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)
Fetching list of newspapers from s3://passim-rebuilt
passim-rebuilt contains 47 newspapers
s3://passim-rebuilt contains 1876 .bz2 files
Joining the two dataframes
Joining the two dataframes... done


In [54]:
issues_to_ingest

,in_rebuilt,in_canonical,newspaper_id,year
id,,,,
EXP-2005-10-04-a,True,NaN,EXP,2005
EXP-2013-08-15-a,True,NaN,EXP,2013
IMP-1900-12-23-a,True,NaN,IMP,1900
JDG-1994-09-27-a,True,NaN,JDG,1994


In [48]:
def l_configure_rebuild(
    canonical_bucket_name: str,
    rebuilt_bucket_name: str,
    issues_to_rebuild: pd.DataFrame = None
) -> list:
    """Generate the config file for data rebuild in a data-driven fashion.

    The configuration file is generated by comparing the newspaper issue IDs
    contained in the s3 canonical bucket against those in the s3 rebuilt
    bucket.
    """
    config = []

    if issues_to_rebuild is None:
        issues_to_ingest, issues_to_rebuild = sync_rebuilt(
            canonical_bucket_name,
            rebuilt_bucket_name
        )

    print((
        f'Watch out: there are {issues_to_rebuild.shape[0]} issues '
        f'that are already ingested ({canonical_bucket_name}) but not '
        f'yet rebuilt ({rebuilt_bucket_name}).'
    ))

    for key, group in issues_to_rebuild.groupby(by='newspaper_id'):
        missing_years = sorted(set(group.year))
        config.append(
            {
                key: [min(missing_years), max(missing_years) + 1]
            }
        )
    return config

In [49]:
rebuild_config = l_configure_rebuild(
    s3_canonical,
    s3_rebuilt,
    issues_to_rebuild=issues_to_rebuild
)

Watch out: there are 161104 issues that are already ingested (s3://original-canonical-staging) but not yet rebuilt (s3://passim-rebuilt).


In [50]:
rebuild_config

[{'BLB': [1845, 1848]},
 {'BNN': [1885, 1893]},
 {'CON': [1902, 1904]},
 {'DFS': [1814, 1850]},
 {'DTT': [1935, 1979]},
 {'DVF': [1878, 1886]},
 {'EDA': [1844, 1845]},
 {'EXP': [1888, 2018]},
 {'EZR': [1806, 1866]},
 {'FCT': [1955, 1964]},
 {'FZG': [1864, 2007]},
 {'GAV': [1855, 1897]},
 {'GAZ': [1842, 1848]},
 {'HRV': [1799, 1802]},
 {'LAB': [1847, 1861]},
 {'LLE': [1871, 2013]},
 {'LLS': [1906, 1999]},
 {'LSE': [1959, 1960]},
 {'MGS': [1841, 1844]},
 {'NTS': [1856, 1874]},
 {'NZG': [1846, 1892]},
 {'NZZ': [1950, 1951]},
 {'OIZ': [1901, 1993]},
 {'SAX': [1882, 1883]},
 {'SDT': [1909, 1998]},
 {'SGZ': [1831, 1882]},
 {'SMZ': [1902, 2002]},
 {'SRT': [1798, 1799]},
 {'VDR': [1844, 1845]},
 {'VHT': [1904, 2005]},
 {'WHD': [1835, 1859]},
 {'ZBT': [1865, 1869]},
 {'arbeitgeber': [1908, 1997]},
 {'handelsztg': [1884, 1885]},
 {'indeplux': [1934, 1935]},
 {'luxembourg1935': [1940, 1941]},
 {'luxwort': [1852, 1951]},
 {'tageblatt': [1913, 1951]}]

In [52]:
ingestion_config = configure_ingestion(
    S3_CANONICAL_DATA_BUCKET,
    S3_REBUILT_DATA_BUCKET,
    issues_to_ingest=issues_to_ingest
)

Watch out: there are 4 issues that are already rebuilt (s3://canonical-rebuilt) but not yet ingested (s3://original-canonical-fixed).


In [53]:
ingestion_config

[{'EXP': [2005, 2014]}, {'IMP': [1900, 1901]}, {'JDG': [1994, 1995]}]

In [51]:
import json
cfg_path = '/home/romanell/Documents/impresso/impresso-processing/rebuild/passim-20191030.json'
with open(cfg_path, 'w') as cfg_file:
    json.dump(rebuild_config, cfg_file, indent=4)

In [173]:
newspapers_to_ingest = set([
    list(x.keys())[0] for x in ingestion_config
])

In [174]:
newspapers_to_ingest

{'BLB',
 'BNN',
 'DFS',
 'DVF',
 'FZG',
 'HRV',
 'LAB',
 'LLE',
 'MGS',
 'NTS',
 'NZG',
 'NZZ',
 'SGZ',
 'SRT',
 'WHD',
 'ZBT'}

In [175]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client -

## Sync with DB

In [19]:
from sanity_check.contents.sync import configure_db_ingestion

In [20]:
s3_canonical_bucket = "s3://original-canonical-staging/"

In [21]:
config = configure_db_ingestion(s3_canonical_bucket, 'sandbox')

Fetching list of newspapers from s3://original-canonical-staging/
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging/ contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)
Connecting to MySQL DB sandbox
Fetched 330079 issue IDs from DB


/home/romanell/Documents/impresso/impresso-data-sanitycheck/sanity_check/contents/sync.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lambda i: i.split('-')[0]


There are 274393 issues from s3://original-canonical-staging/ missing from MySQL (sandbox).


/home/romanell/Documents/impresso/impresso-data-sanitycheck/sanity_check/contents/sync.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lambda i: int(i.split('-')[1])


In [22]:
config

[{'BDC': [1839, 1840]},
 {'BLB': [1845, 1846]},
 {'BLB': [1846, 1847]},
 {'BLB': [1847, 1848]},
 {'BNN': [1885, 1886]},
 {'BNN': [1886, 1887]},
 {'BNN': [1887, 1888]},
 {'BNN': [1888, 1889]},
 {'BNN': [1889, 1890]},
 {'BNN': [1890, 1891]},
 {'BNN': [1891, 1892]},
 {'BNN': [1892, 1893]},
 {'CDV': [1843, 1844]},
 {'CDV': [1844, 1845]},
 {'CDV': [1849, 1850]},
 {'CDV': [1850, 1851]},
 {'CDV': [1851, 1852]},
 {'CDV': [1852, 1853]},
 {'CDV': [1853, 1854]},
 {'CDV': [1854, 1855]},
 {'CDV': [1855, 1856]},
 {'CDV': [1856, 1857]},
 {'CDV': [1857, 1858]},
 {'CON': [1902, 1903]},
 {'CON': [1903, 1904]},
 {'DFS': [1814, 1815]},
 {'DFS': [1815, 1816]},
 {'DFS': [1816, 1817]},
 {'DFS': [1817, 1818]},
 {'DFS': [1818, 1819]},
 {'DFS': [1819, 1820]},
 {'DFS': [1820, 1821]},
 {'DFS': [1821, 1822]},
 {'DFS': [1822, 1823]},
 {'DFS': [1823, 1824]},
 {'DFS': [1824, 1825]},
 {'DFS': [1825, 1826]},
 {'DFS': [1826, 1827]},
 {'DFS': [1827, 1828]},
 {'DFS': [1828, 1829]},
 {'DFS': [1829, 1830]},
 {'DFS': [1830, 

In [24]:
cluster.close()